In [110]:
import pandas as pd
import numpy as np
import time
import re

In [111]:
cpu = pd.read_csv('../CPU_final.csv')

In [112]:
col = 'memory_type'
cpu[col].unique()

array(['DDR5', 'DDR5, DDR4', 'DDR4', 'DDR4, DDR3L', nan, 'DDR3',
       'DDR3, DDR2'], dtype=object)

In [113]:
mt_bitmask = ['DDR5', 'DDR4', 'DDR3L', 'DDR3', 'DDR2']

In [114]:
for i in range(len(cpu[col])):
    if pd.isnull(cpu[col][i]): continue
    mt_bit = 0
    for j in range(len(mt_bitmask)):
        if mt_bitmask[j] in cpu.loc[i, col]:
            mt_bit |= 1 << j
    cpu.loc[i, col] = mt_bit

In [115]:
cpu[col].unique()

array([1, 3, 2, 14, nan, 8, 24], dtype=object)

In [116]:
for col in cpu.columns:
    for i in range(len(cpu[col])):
        if pd.isnull(cpu[col][i]): continue
        if '최대' in str(cpu.loc[i, col]):
            print(col)

memory_clock
memory_clock
memory_clock


In [117]:
col = 'memory_clock'
cpu[col].unique()

array(['5200', '4800, 3200', '3200', '5600, 3200', '2933', '2666', '2400',
       nan, '최대 2133', '2667', '2133', '최대 2400', '1333', '1066', '2000',
       '800', '최대 1866'], dtype=object)

In [118]:
for i in range(len(cpu[col])):
    if pd.isnull(cpu[col][i]): continue
    if '최대 ' in cpu.loc[i, col]:
        cpu.loc[i, col] = cpu.loc[i, col].replace('최대 ', '')

In [119]:
col = 'core_number'
cpu[col].unique()

array(['6', '8', '6+4', '8+16', '6+8', '8+8', '8+4', '4', '16', '12', '2',
       '64', '32', '24', '18', '10', '14', '10 ', '28', '14 ', '20', '22',
       '3'], dtype=object)

In [120]:
for i in range(len(cpu[col])):
    if pd.isnull(cpu[col][i]): continue
    cpu.loc[i, col] = str(eval(cpu.loc[i, col]))

In [121]:
cpu[col].unique()

array(['6', '8', '10', '24', '14', '16', '12', '4', '2', '64', '32', '18',
       '28', '20', '22', '3'], dtype=object)

In [122]:
col = 'thread_number'
cpu[col].unique()

array(['12', '16', '12+4', '16+16', '12+8', '16+8', '16+4', '8', '32',
       '24', '4', '6', '2', '128', '64', '48', '36', '20', '28', '56',
       '40', '44', '3'], dtype=object)

In [123]:
for i in range(len(cpu[col])):
    if pd.isnull(cpu[col][i]): continue
    cpu.loc[i, col] = str(eval(cpu.loc[i, col]))
cpu[col].unique()

array(['12', '16', '32', '20', '24', '8', '4', '6', '2', '128', '64',
       '48', '36', '28', '56', '40', '44', '3'], dtype=object)

In [124]:
col = 'tdp'
print(cpu[col].unique())
for i in range(len(cpu[col])):
    if pd.isnull(cpu[col][i]): continue
    if '~' in cpu.loc[i, col]:
        cpu.loc[i, col] = cpu.loc[i, col][cpu.loc[i, col].index('~')+1:]
print(cpu[col].unique())

['65' '120' nan '105' '65~117' '65~180' '60~89' '170' '46' '125~150'
 '125~190' '125~241' '280' '58' '65~202' '165' '125' '95' '150~241' '73'
 '54' '72' '85' '250' '180' '84' '35' '71' '51' '88' '77' '55' '100' '91'
 '205' '80' '185' '150' '53' '135' '140' '69' '130' '225' '155' '127'
 '115']
['65' '120' nan '105' '117' '180' '89' '170' '46' '150' '190' '241' '280'
 '58' '202' '165' '125' '95' '73' '54' '72' '85' '250' '84' '35' '71' '51'
 '88' '77' '55' '100' '91' '205' '80' '185' '53' '135' '140' '69' '130'
 '225' '155' '127' '115']


In [125]:
cpu = cpu.rename(columns={'pbtmtp': 'pbpmtp'})

In [126]:
col = 'pbpmtp'
print(cpu[col].unique())
for i in range(len(cpu[col])):
    if pd.isnull(cpu[col][i]): continue
    if '~' in cpu.loc[i, col]:
        cpu.loc[i, col] = cpu.loc[i, col][cpu.loc[i, col].index('~')+1:]
print(cpu[col].unique())

[nan '65~148' '125~253' '125~181' '65~154' '65~219' '150~253' '60~89'
 '58~89' '35~106' '35~92' '35~69']
[nan '148' '253' '181' '154' '219' '89' '106' '92' '69']


In [127]:
col = 'has_graphic'
cpu[col] = np.where(pd.isnull(cpu['graphic_name']), 0, 1)

In [128]:
col = 'memory_capacity'
for i in range(len(cpu[col])):
    if pd.isnull(cpu[col][i]): continue
    if 'TB' in cpu.loc[i, col]:
        tb = float(cpu.loc[i, col].replace('TB', ''))
        cpu.loc[i, col] = f'{tb * 1024:.0f}'

In [129]:
cpu[col].unique()

array(['128', nan, '64', '2048', '256', '1024', '4096', '6144', '4608'],
      dtype=object)

In [130]:
col = 'memory_clock'
print(cpu[col].unique())
for i in range(len(cpu[col])):
    if pd.isnull(cpu[col][i]): continue
    if ',' in cpu.loc[i, col]:
        cpu.loc[i, col] = cpu.loc[i, col][:cpu.loc[i, col].index(',')]
print(cpu[col].unique())

['5200' '4800, 3200' '3200' '5600, 3200' '2933' '2666' '2400' nan '2133'
 '2667' '1333' '1066' '2000' '800' '1866']
['5200' '4800' '3200' '5600' '2933' '2666' '2400' nan '2133' '2667' '1333'
 '1066' '2000' '800' '1866']


In [132]:
col = 'pcie_version'
print(cpu[col].unique())
pv_bitmask = ['5.0', '4.0', '3.0']
for i in range(len(cpu[col])):
    if pd.isnull(cpu[col][i]): continue
    pv_bit = 0
    for j in range(len(pv_bitmask)):
        if pv_bitmask[j] in cpu.loc[i, col]:
            pv_bit |= 1 << j
    cpu.loc[i, col] = pv_bit
print(cpu[col].unique())

['PCIe5.0' 'PCIe5.0, 4.0' 'PCIe3.0' 'PCIe4.0' nan]
[1 3 4 2 nan]


In [135]:
col = 'clock_basic'
print(cpu[col].unique())
for i in range(len(cpu[col])):
    if pd.isnull(cpu[col][i]): continue
    if ' ~' in cpu.loc[i, col]:
        cpu.loc[i, col] = cpu.loc[i, col][:cpu.loc[i, col].index(' ~')]
print(cpu[col].unique())

['3.8' '4.2' '2.5' '3.7' '4.7' '3.9' '3.0' '3.5' '3.4' '4.5' '2.1' '3.3'
 '3.2' '2.9' '2.80' '2.0' '4.4' '3.6' '2.6' '2.7' '4.1' '2.4' '4.0' '3.20'
 '3.70' '2.8' '3.1' '3.60' '4.3' '2.2' '1.1' '2.45' '1.4' '3.5 ' '2.66'
 '2.50' '2.10' '3.16' '1.70' '2.93' '2.20' '3.0 ' '2.3' '2.60' '3.80'
 '2.33' '2.40' '1.90' '2.30' '2.5 ' '2.83' '3.06' '3.33' '2.25' '1.8'
 '4.00' '2.5 ~ 2.99' '2.35' '1.7' '2.53' '2.00' '3.40' '1.6' '2.90' '3.50']
['3.8' '4.2' '2.5' '3.7' '4.7' '3.9' '3.0' '3.5' '3.4' '4.5' '2.1' '3.3'
 '3.2' '2.9' '2.80' '2.0' '4.4' '3.6' '2.6' '2.7' '4.1' '2.4' '4.0' '3.20'
 '3.70' '2.8' '3.1' '3.60' '4.3' '2.2' '1.1' '2.45' '1.4' '3.5 ' '2.66'
 '2.50' '2.10' '3.16' '1.70' '2.93' '2.20' '3.0 ' '2.3' '2.60' '3.80'
 '2.33' '2.40' '1.90' '2.30' '2.5 ' '2.83' '3.06' '3.33' '2.25' '1.8'
 '4.00' '2.35' '1.7' '2.53' '2.00' '3.40' '1.6' '2.90' '3.50']


In [137]:
col = 'clock_max'
print(cpu[col].unique())
for i in range(len(cpu[col])):
    if pd.isnull(cpu[col][i]): continue
    if '~' in cpu.loc[i, col]:
        cpu.loc[i, col] = cpu.loc[i, col][cpu.loc[i, col].index('~ ')+1:]
    if '/' in cpu.loc[i, col]:
        cpu.loc[i, col] = cpu.loc[i, col][cpu.loc[i, col].index('/ ')+1:]
print(cpu[col].unique())

['5.1' '5.0' '4.6' '5.3' '4.4' '5.8' '5.4' '4.8' '4.2' '5.2' '4.9' '4.3'
 '5.7' '4.7' '6.0' '4.5' '4.0' '5.6' nan '4.00' '4.1' '4' '4.60' '4.70'
 '5.5' '5.00' '3.7' '4.50' '3.9' '3.1' '4.25' '3.6' '3.8' '3.4' '3.46'
 '3.2' '3.5' '3.3' '4.35' ' 3.49' '3.90' '3.70' '3.20' '3.00' '1.90'
 '3.35' '1.7' '3.33' '3.7  / 3.8' '2.66' '4.30']
['5.1' '5.0' '4.6' '5.3' '4.4' '5.8' '5.4' '4.8' '4.2' '5.2' '4.9' '4.3'
 '5.7' '4.7' '6.0' '4.5' '4.0' '5.6' nan '4.00' '4.1' '4' '4.60' '4.70'
 '5.5' '5.00' '3.7' '4.50' '3.9' '3.1' '4.25' '3.6' '3.8' '3.4' '3.46'
 '3.2' '3.5' '3.3' '4.35' ' 3.49' '3.90' '3.70' '3.20' '3.00' '1.90'
 '3.35' '1.7' '3.33' ' 3.8' '2.66' '4.30']


In [138]:
today = time.strftime('%y%m%d')
pd.DataFrame(cpu).to_csv(f'../part_cpu{today}.csv', index=None)